In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bengali-news/article.csv
/kaggle/input/bengali-news/summary.csv


In [10]:
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from transformers import MT5ForConditionalGeneration, AutoTokenizer ,DataCollatorForSeq2Seq, Trainer, TrainingArguments
import os
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast


In [11]:
import pandas as pd

# Load the article and summary CSV files without headers
articles_df = pd.read_csv('/kaggle/input/bengali-news/article.csv', header=None)
summaries_df = pd.read_csv('/kaggle/input/bengali-news/summary.csv', header=None)

# Assign column names manually
articles_df.columns = ['article']
summaries_df.columns = ['summary']

# Concatenate them into a single DataFrame for ease of processing
df = pd.concat([articles_df, summaries_df], axis=1)

# Display the first few rows for verification
print(df.head())


                                             article  \
0  স্ট্যান্ডার্ড চার্টার্ড ব্যাংকের নতুন প্রধান ন...   
1  রাজধানী থেকে চামড়া শিল্পগুলো সাভারে স্থানান্তর...   
2  দেশীয় শিল্প বিকাশে সরকারের সব ধরনের উদ্যোগ অব্...   
3  একীভূত হতে চলেছে অনলাইনে শ্রেণিবদ্ধ বিজ্ঞাপন স...   
4  যাত্রীবাহী একটি বাসে আগুন দেওয়ার আধা ঘণ্টার মধ...   

                                             summary  
0          স্ট্যান্ডার্ড চার্টার্ডের নতুন সিইও আবরার  
1  মার্চের মধ্যে সাভারে চামড়া শিল্পের সিইটিপি: মন...  
2                       ওয়ালটন কারখানায় শিল্পমন্ত্রী  
3                    একীভূত হচ্ছে এখানেই ডটকমওএলএক্স  
4              বাসে আগুন: নড়াইলের পৌর মেয়র গ্রেপ্তার  


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19096 entries, 0 to 19095
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   article  19096 non-null  object
 1   summary  19096 non-null  object
dtypes: object(2)
memory usage: 298.5+ KB


In [13]:
df

,article,summary
0,স্ট্যান্ডার্ড চার্টার্ড ব্যাংকের নতুন প্রধান ন...,স্ট্যান্ডার্ড চার্টার্ডের নতুন সিইও আবরার
1,রাজধানী থেকে চামড়া শিল্পগুলো সাভারে স্থানান্তর...,মার্চের মধ্যে সাভারে চামড়া শিল্পের সিইটিপি: মন...
2,দেশীয় শিল্প বিকাশে সরকারের সব ধরনের উদ্যোগ অব্...,ওয়ালটন কারখানায় শিল্পমন্ত্রী
3,একীভূত হতে চলেছে অনলাইনে শ্রেণিবদ্ধ বিজ্ঞাপন স...,একীভূত হচ্ছে এখানেই ডটকমওএলএক্স
4,যাত্রীবাহী একটি বাসে আগুন দেওয়ার আধা ঘণ্টার মধ...,বাসে আগুন: নড়াইলের পৌর মেয়র গ্রেপ্তার
...,...,...
19091,মাঝারি থেকে দূরপাল্লার ক্ষেপণাস্ত্রের সফল পরীক...,যেকোনও সময় ফের ক্ষেপণাস্ত্র পরীক্ষার হুমকি উত্...
19092,বিশ্বকবি রবীন্দ্রনাথ ঠাকুরের ১৫৬তম ও জাতীয় কবি...,আবুধাবিতে রবীন্দ্রনজরুল জন্মজয়ন্তী পালন
19093,এশিয়ান কন্টিনেন্টাল দাবা চ্যাম্পিয়নশিপের চতুর্...,চীনে রাজীব ও লিজার ড্র
19094,শাখার পূর্ণাঙ্গ কমিটি করতে না পারার জন্য বড় ভা...,বড় ভাইদের দাবি মেটাতে হিমশিম ছাত্রলীগ


In [14]:
df.dropna()

,article,summary
0,স্ট্যান্ডার্ড চার্টার্ড ব্যাংকের নতুন প্রধান ন...,স্ট্যান্ডার্ড চার্টার্ডের নতুন সিইও আবরার
1,রাজধানী থেকে চামড়া শিল্পগুলো সাভারে স্থানান্তর...,মার্চের মধ্যে সাভারে চামড়া শিল্পের সিইটিপি: মন...
2,দেশীয় শিল্প বিকাশে সরকারের সব ধরনের উদ্যোগ অব্...,ওয়ালটন কারখানায় শিল্পমন্ত্রী
3,একীভূত হতে চলেছে অনলাইনে শ্রেণিবদ্ধ বিজ্ঞাপন স...,একীভূত হচ্ছে এখানেই ডটকমওএলএক্স
4,যাত্রীবাহী একটি বাসে আগুন দেওয়ার আধা ঘণ্টার মধ...,বাসে আগুন: নড়াইলের পৌর মেয়র গ্রেপ্তার
...,...,...
19091,মাঝারি থেকে দূরপাল্লার ক্ষেপণাস্ত্রের সফল পরীক...,যেকোনও সময় ফের ক্ষেপণাস্ত্র পরীক্ষার হুমকি উত্...
19092,বিশ্বকবি রবীন্দ্রনাথ ঠাকুরের ১৫৬তম ও জাতীয় কবি...,আবুধাবিতে রবীন্দ্রনজরুল জন্মজয়ন্তী পালন
19093,এশিয়ান কন্টিনেন্টাল দাবা চ্যাম্পিয়নশিপের চতুর্...,চীনে রাজীব ও লিজার ড্র
19094,শাখার পূর্ণাঙ্গ কমিটি করতে না পারার জন্য বড় ভা...,বড় ভাইদের দাবি মেটাতে হিমশিম ছাত্রলীগ


In [15]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.30, shuffle=True)


In [16]:
from datasets import Dataset
ds_train = Dataset.from_pandas(df_train)
ds_test = Dataset.from_pandas(df_test)

In [17]:
model_name = "facebook/mbart-large-50-many-to-many-mmt"

tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [18]:
def tokenize_data(data):
    input_features = tokenizer(data['article'], truncation=True, max_length=1024)
    label = tokenizer(data['summary'],truncation =True, max_length=128)
    return {
        'input_ids':input_features['input_ids'],
        'attention_mask': input_features['attention_mask'],
        'labels': label['input_ids'],
    }


In [19]:
tokenized_ds_train = ds_train.map(
    tokenize_data,
    batched=True,
    remove_columns=ds_train.column_names  # Remove original columns
)

# Apply tokenization to the test dataset
tokenized_ds_test = ds_test.map(
    tokenize_data,
    batched=True,
    remove_columns=ds_test.column_names
)

Map:   0%|          | 0/13367 [00:00<?, ? examples/s]

Map:   0%|          | 0/5729 [00:00<?, ? examples/s]

In [20]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer = tokenizer,
    model = model,
    return_tensors = 'pt'
)

In [21]:
from transformers import Seq2SeqTrainingArguments

#model.gradient_checkpointing_enable()

training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps = 8,
    weight_decay=0.01,
    save_total_limit=1,
    num_train_epochs=1,
    fp16=True,
    predict_with_generate=False,
    logging_dir='./logs',
    logging_steps=10,
    generation_max_length = 128,
    push_to_hub = False,
    dataloader_num_workers=1,
)

In [22]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds_train,
    eval_dataset=tokenized_ds_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [23]:
import wandb
wandb.disabled = True
os.environ['WANDB_DISABLED'] = 'true'

In [24]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all in

Epoch,Training Loss,Validation Loss
0,2.929000,2.898719


/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disab

TrainOutput(global_step=417, training_loss=3.4192326268989692, metrics={'train_runtime': 2709.5004, 'train_samples_per_second': 4.933, 'train_steps_per_second': 0.154, 'total_flos': 1743422036901888.0, 'train_loss': 3.4192326268989692, 'epoch': 0.9982046678635548})

In [25]:
import gc
import torch

# Assuming `obj` is the object consuming GPU memory
obj = None

# Collect garbage
gc.collect()

# Empty PyTorch cache
torch.cuda.empty_cache()

In [26]:
pip install rouge_score

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=46545c0335b390c235f2c29318e4c8412da2e742bbcc5558bbee6ce6207704e2
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.0 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


# Evaluate ROUGE scores

In [28]:
import evaluate
import numpy as np
from nltk.tokenize import RegexpTokenizer

rouge_metric = evaluate.load("rouge")

# define function for custom tokenization
def tokenize_sentence(arg):
    encoded_arg = tokenizer(arg)
    return tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

# define function to get ROUGE scores with custom tokenization
def metrics_func(eval_arg):
    preds, labels = eval_arg

    # Replace -100
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Convert id tokens to text

    text_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    text_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Insert a line break (\n) in each sentence for ROUGE scoring

    # (Note : Please change this code, when you perform on other languages except for Bengali)
    text_preds = [(p if p.endswith(("!", "!", "?", "?", "।")) else p + "।") for p in text_preds]
    text_labels = [(l if l.endswith(("!", "!", "?", "?", "।")) else l + "।") for l in text_labels]
    sent_tokenizer_bn = RegexpTokenizer(u'[^!!??।]*[!!??।]')
    text_preds = ["\n".join(np.char.strip(sent_tokenizer_bn.tokenize(p))) for p in text_preds]
    text_labels = ["\n".join(np.char.strip(sent_tokenizer_bn.tokenize(l))) for l in text_labels]

    # compute ROUGE score with custom tokenization
    return rouge_metric.compute(
    predictions=text_preds,
    references=text_labels,
    tokenizer=tokenize_sentence
    )

In [29]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [30]:
from torch.utils.data import DataLoader

sample_dataloader = DataLoader(
    tokenized_ds_test.with_format("torch"),
    collate_fn=data_collator,
    batch_size=1
)

for batch in sample_dataloader:
    with torch.no_grad():
        preds = model.generate(
            batch["input_ids"].to(device),
            num_beams=2,
            num_return_sequences=1,
            no_repeat_ngram_size=1,
            remove_invalid_values=True,
            max_length=128
        )
    labels = batch["labels"]
    break

metrics_func([preds, labels])

/opt/conda/lib/python3.10/site-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


{'rouge1': 0.20979020979020976,
 'rouge2': 0.14184397163120568,
 'rougeL': 0.19580419580419578,
 'rougeLsum': 0.19310344827586207}